# Value Iteration example

### Sources:
- http://www.cs.uu.nl/docs/vakken/b2ki/LastYear/Docs/Slides/mdps-handouts.pdf

In [ ]:
from typing import Tuple, List
import numpy as np

In [ ]:
class Maze:
    def __init__(self, R: np.ndarray, end: List[Tuple[int, int]],
                 noise: float = 0.7, discount: float = 0.9, threshold: float = 0.5):
        self.R = R
        self.utility = np.zeros(R.shape)
        self.end_states = end

        self.noise = noise
        self.discount = discount
        self.threshold = threshold
    
    def _calc_new_value(self):
        ...
    
    def _get_action_positions(self, current_pos: Tuple[int, int]):
        """"""
        row, col = current_pos

        up = (row - 1, col) if row - 1 >= 0 else current_pos
        right = (row, col + 1) if col + 1 <= self.R.shape[1] else current_pos
        left = (row, col -1) if col - 1 >= 0 else current_pos
        down = (row + 1, col) if row + 1 <= self.R.shape[0] else current_pos
        return up, right, left, down
    
    def iterate_values(self, noise: float = 0.7):
        """Value iteration method"""
        delta = 0
        new_utility = np.zeros(self.utility.shape)
        # get all positions in the grid
        positions = [(i,j) for i in range(self.R.shape[0]) for j in range(self.R.shape[1])]
        
        while delta < self.threshold:
            for pos in positions:
                # check if we are evaluating an end state
                if pos in self.end_states:
                    continue

                # save the current value
                value = self.utility[pos]
                # 
                actions = self._get_action_positions(pos)
                print(pos, actions)
#                 for index, action in enumerate(actions):
#                     print(index, action)
#                     noise_actions = [x for x in actions if x != action]
#                 print()
                    
                    
                
                
            print(new_utility)
            break
        
        # delta = max(delta, np.amax(new_utility))
        
        
        

In [ ]:
class Agent:
    def __init__(self, pos: Tuple[int, int], model: Maze):
        self.pos = pos
        self.env = model

In [ ]:
start_position = (3, 2)
finish_positions = [(0, 3), (3, 0)]

rewards = np.array([[-1, -1, -1, 40],
                    [-1, -1, -10, -10],
                    [-1, -1, -1, -1],
                    [10, -2, 0, -1]])

# build maze and agent
maze = Maze(rewards, finish_positions)
agent = Agent(start_position, maze)

maze.iterate_values()